In [1]:
# 데이터 제작, 앞 예제 같이 글자 학습, 원-핫 인코딩
# 알파벳과 함글 나열 뒤 함글자씩 배열에 넣는다.
# 넣는 글자는 연관 배열(키/값 쌍)dmfh qusrud
import tensorflow as tf
import numpy as np

In [2]:
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], ['girl', '소녀'], ['kiss', '키스'],['love', '사랑']]

In [3]:
# 입력 단어, 출력단어 한 글자씩 떼서 배열로 만든 후 원-핫 인코딩 유틸리티 함수 제작
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        # 인코더 셀의 입력값을 위해 입력 단어를 한글자씩 떼어 배열로 만든다.
        input = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값을 위해 출력 단어의 글자들을 배열로 만들고, 시작은 나타내는 심볼 'S'를 맨 앞에 붙입니다.
        output = [num_dic[n] for n in ('S' + seq[1])]
        # 학습을 위해 비교할 디코더 셀의 출력값을 만들고, 출력의 끝을 알려주는 심볼 'E'를 마지막에 붙입니다.
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
        
    return input_batch, output_batch, target_batch

In [4]:
# 학습 수치 정의
learnging_rate = 0.01
n_hidden = 128
total_epoch = 100

n_class = n_input = dic_len
# 인코더의 입력값, 디코더의 입력값, 출력값에 쓸 플레이스 홀더 구성
# 입력은 [batch size, time steps, input size] 형식
# 출력은 [batch size, time steps] 형식

In [5]:
# 신경망 구성 다른 글자수의 것도 할 수 있게 일단 None, 단, 같은 배치때 입력되는 데이터는 글자수, 단계가 모두 같아야 한다.
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

In [6]:
# RNN 모델을 위한 셀 구성
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)
    
# 디코더에 초기상대값아닌 인코더 최종값을 넣어야 한다.
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state = enc_states, dtype = tf.float32)
# 인코더의 계산한 상태를 디코더로 전파 가능 initial_state = enc_states 옵션 쓰면 간단

W0808 15:07:33.009679 15436 deprecation.py:323] From <ipython-input-6-7e1ad10aaf9d>:3: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
W0808 15:07:33.026890 15436 deprecation.py:323] From <ipython-input-6-7e1ad10aaf9d>:6: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0808 15:07:33.120697 15436 deprecation.py:506] From c:\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructo

In [7]:
# 출력층 제작, 손실 함수와 최적화 함수 구성
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learnging_rate).minimize(cost)

W0808 15:10:15.167797 15436 deprecation.py:323] From <ipython-input-7-281f6046d12f>:2: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [11]:
# 학습을 시키는 코드는 앞과 같아
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict = {enc_input: input_batch, dec_input:output_batch, targets:target_batch})
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    
print('최적화 완료!')

Epoch: 0001 cost = 3.721476
Epoch: 0002 cost = 2.690071
Epoch: 0003 cost = 1.692258
Epoch: 0004 cost = 1.471586
Epoch: 0005 cost = 0.767640
Epoch: 0006 cost = 0.542222
Epoch: 0007 cost = 0.668714
Epoch: 0008 cost = 0.242237
Epoch: 0009 cost = 0.241188
Epoch: 0010 cost = 0.103662
Epoch: 0011 cost = 0.180770
Epoch: 0012 cost = 0.332960
Epoch: 0013 cost = 0.166649
Epoch: 0014 cost = 0.041671
Epoch: 0015 cost = 0.092945
Epoch: 0016 cost = 0.085520
Epoch: 0017 cost = 0.029800
Epoch: 0018 cost = 0.019691
Epoch: 0019 cost = 0.058325
Epoch: 0020 cost = 0.125733
Epoch: 0021 cost = 0.018935
Epoch: 0022 cost = 0.012139
Epoch: 0023 cost = 0.018184
Epoch: 0024 cost = 0.033796
Epoch: 0025 cost = 0.029963
Epoch: 0026 cost = 0.021377
Epoch: 0027 cost = 0.008317
Epoch: 0028 cost = 0.005491
Epoch: 0029 cost = 0.005918
Epoch: 0030 cost = 0.004522
Epoch: 0031 cost = 0.003950
Epoch: 0032 cost = 0.005570
Epoch: 0033 cost = 0.003715
Epoch: 0034 cost = 0.005074
Epoch: 0035 cost = 0.003497
Epoch: 0036 cost = 0

In [13]:
# 결과를 확인하기 위해 단어를 입력받아 번역 단어를 예측하는 함수 제작
def translate(word):
    seq_data = [word, 'P' * len(word)]
    
    input_batch, output_batch, target_batch = make_batch([seq_data])
    # 예측 모델을 돌립니다. 세 번째 차원을 argmax로 취해 가장 확률이 높은 글자를 예측값으로 만듭니다.
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    
    decoded = [char_arr[i] for i in result[0]]
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    
    return translated

In [17]:
print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))
print('xcvf ->', translate('xcvf'))


=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 
xcvf -> 사랑이랑
